<a href="https://colab.research.google.com/github/kviercz/AAI-511_Group-Project/blob/main/Base_composers_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from google.colab import drive
# Run this
drive.mount('/content/drive/')
dataset_path = '/content/drive/MyDrive/DataFiles/'
os.chdir(dataset_path)

Mounted at /content/drive/


In [3]:
!pip install pretty_midi mido

import pretty_midi
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from mido import KeySignatureError
import os
import random

# --- Data Loading and Preprocessing ---

def load_midi_features(directory):
    """Loads MIDI files and extracts pitch, duration, and velocity features."""
    pitch_sequences = []
    duration_sequences = []
    velocity_sequences = []

    for filename in os.listdir(directory):
        if filename.endswith(".mid"):
            midi_path = os.path.join(directory, filename)
            try:
                midi = pretty_midi.PrettyMIDI(midi_path)
                for instrument in midi.instruments:
                    pitches = [note.pitch for note in instrument.notes]
                    durations = [note.end - note.start for note in instrument.notes]
                    velocities = [note.velocity for note in instrument.notes]
                    pitch_sequences.append(pitches)
                    duration_sequences.append(durations)
                    velocity_sequences.append(velocities)
            except KeySignatureError as e:
                print(f"Skipping file {filename} due to KeySignatureError: {e}")

    return pitch_sequences, duration_sequences, velocity_sequences

# Load features for each composer
bach_pitches, bach_durations, bach_velocities = load_midi_features("/content/drive/MyDrive/DataFiles/Bach/")
mozart_pitches, mozart_durations, mozart_velocities = load_midi_features("/content/drive/MyDrive/DataFiles/Mozart/")
beethoven_pitches, beethoven_durations, beethoven_velocities = load_midi_features("/content/drive/MyDrive/DataFiles/Beethoven/")
chopin_pitches, chopin_durations, chopin_velocities = load_midi_features("/content/drive/MyDrive/DataFiles/Chopin/")

def normalize_features(pitch_sequences, duration_sequences, velocity_sequences):
    """Normalizes pitches, durations, and velocities to a 0-1 range."""
    normalized_pitches = [[note / 127.0 for note in sequence] for sequence in pitch_sequences]
    max_duration = max([max(durations) for durations in duration_sequences if durations])  # Find max duration
    normalized_durations = [[duration / max_duration for duration in sequence] for sequence in duration_sequences]
    normalized_velocities = [[velocity / 127.0 for velocity in sequence] for sequence in velocity_sequences]

    return normalized_pitches, normalized_durations, normalized_velocities

# Normalize the features
bach_pitches, bach_durations, bach_velocities = normalize_features(bach_pitches, bach_durations, bach_velocities)
mozart_pitches, mozart_durations, mozart_velocities = normalize_features(mozart_pitches, mozart_durations, mozart_velocities)
beethoven_pitches, beethoven_durations, beethoven_velocities = normalize_features(beethoven_pitches, beethoven_durations, beethoven_velocities)
chopin_pitches, chopin_durations, chopin_velocities = normalize_features(chopin_pitches, chopin_durations, chopin_velocities)



# --- Data Balancing ---

# min_samples = min(len(bach_data), len(mozart_data), len(beethoven_data), len(chopin_data))

# balanced_bach = random.sample(bach_data, min_samples)
# balanced_mozart = random.sample(mozart_data, min_samples)
# balanced_beethoven = random.sample(beethoven_data, min_samples)
# balanced_chopin = random.sample(chopin_data, min_samples)

# --- Data Augmentation ---

def augment_sequence(sequence, num_augmentations=2):
    """Augments a note sequence by transposing and creating variations."""
    augmented_sequences = [sequence]
    for _ in range(num_augmentations):
        transpose_amount = random.randint(-5, 5)  # Transpose by up to 5 semitones
        augmented_sequence = [[note[0] + transpose_amount, note[1], note[2]] for note in sequence] # Transpose only the pitch value (index 0)
        augmented_sequences.append(augmented_sequence)
    return augmented_sequences

def combine_features(pitch_sequences, duration_sequences, velocity_sequences):
    """Combines pitch, duration, and velocity features into a single array."""
    combined_features = []
    for pitches, durations, velocities in zip(pitch_sequences, duration_sequences, velocity_sequences):
        sequence_length = min(len(pitches), len(durations), len(velocities))
        combined_sequence = []
        for i in range(sequence_length):
            combined_sequence.append([pitches[i], durations[i], velocities[i]])
        combined_features.append(combined_sequence)
    return combined_features

# Combine features for each composer
combined_bach = combine_features(bach_pitches, bach_durations, bach_velocities)
combined_mozart = combine_features(mozart_pitches, mozart_durations, mozart_velocities)
combined_beethoven = combine_features(beethoven_pitches, beethoven_durations, beethoven_velocities)
combined_chopin = combine_features(chopin_pitches, chopin_durations, chopin_velocities)

# Augment combined data
augmented_combined_bach = [seq for orig_seq in combined_bach for seq in augment_sequence(orig_seq)]
augmented_combined_mozart = [seq for orig_seq in combined_mozart for seq in augment_sequence(orig_seq)]
augmented_combined_beethoven = [seq for orig_seq in combined_beethoven for seq in augment_sequence(orig_seq)]
augmented_combined_chopin = [seq for orig_seq in combined_chopin for seq in augment_sequence(orig_seq)]

def augment_with_noise(sequence, noise_factor=0.01):
    """Adds noise to the sequence to create variation."""
    noisy_sequence = []
    for note in sequence:
        noisy_note = [
            note[0] + random.uniform(-noise_factor, noise_factor),  # Pitch noise
            note[1] + random.uniform(-noise_factor, noise_factor),  # Duration noise
            note[2] + random.uniform(-noise_factor, noise_factor)   # Velocity noise
        ]
        noisy_sequence.append(noisy_note)
    return noisy_sequence

def augment_with_tempo(sequence, tempo_factor_range=(0.9, 1.1)):
    """Scales the tempo of the sequence."""
    tempo_factor = random.uniform(*tempo_factor_range)
    tempo_scaled_sequence = [[note[0], note[1] * tempo_factor, note[2]] for note in sequence]
    return tempo_scaled_sequence

# Augment with noise and tempo scaling
augmented_combined_bach.extend([augment_with_noise(seq) for seq in augmented_combined_bach])
augmented_combined_bach.extend([augment_with_tempo(seq) for seq in augmented_combined_bach])

augmented_combined_mozart.extend([augment_with_noise(seq) for seq in augmented_combined_mozart])
augmented_combined_mozart.extend([augment_with_tempo(seq) for seq in augmented_combined_mozart])

augmented_combined_beethoven.extend([augment_with_noise(seq) for seq in augmented_combined_beethoven])
augmented_combined_beethoven.extend([augment_with_tempo(seq) for seq in augmented_combined_beethoven])

augmented_combined_chopin.extend([augment_with_noise(seq) for seq in augmented_combined_chopin])
augmented_combined_chopin.extend([augment_with_tempo(seq) for seq in augmented_combined_chopin])

import numpy as np

def create_windows(sequence, window_size, step_size=1):
    """Splits a sequence into overlapping windows."""
    windows = []
    for start in range(0, len(sequence) - window_size + 1, step_size):
        end = start + window_size
        window = sequence[start:end]
        windows.append(window)
    return windows


# Apply windowing to all sequences
window_size = 50  # Choose an appropriate window size
step_size = 10    # Overlapping windows (can be set to 1 for fully overlapping)

# Function to apply windowing to an entire dataset
def apply_windowing(sequences, window_size, step_size=1):
    windowed_sequences = []
    for sequence in sequences:
        windows = create_windows(sequence, window_size, step_size)
        windowed_sequences.extend(windows)  # Add all windows of a sequence
    return windowed_sequences

# Apply windowing to each composer's dataset
windowed_bach = apply_windowing(augmented_combined_bach, window_size, step_size)
windowed_mozart = apply_windowing(augmented_combined_mozart, window_size, step_size)
windowed_beethoven = apply_windowing(augmented_combined_beethoven, window_size, step_size)
windowed_chopin = apply_windowing(augmented_combined_chopin, window_size, step_size)

# Combine windowed data
all_windowed_data = windowed_bach + windowed_mozart + windowed_beethoven + windowed_chopin

# Flatten windowed data to extract unique pitches
# flattened_data = [note for sequence in all_windowed_data for note in sequence]

# Update labels for each windowed sequence
windowed_labels = [0] * len(windowed_bach) + [1] * len(windowed_mozart) + \
                  [2] * len(windowed_beethoven) + [3] * len(windowed_chopin)

# Flatten windowed data to extract unique pitches
flattened_data = [note[0] for sequence in all_windowed_data for note in sequence] # Extract only the pitch value

# Tokenize the note pitches
all_pitches = sorted(set(flattened_data)) # Create a set from the flattened pitches
note_to_int = {note: number for number, note in enumerate(all_pitches)}

# Convert note sequences to integer sequences (using only pitch)
tokenized_data = [[note_to_int[note[0]] for note in sequence] for sequence in all_windowed_data]
# Pad sequences to ensure consistent input shape
X = pad_sequences(tokenized_data, maxlen=window_size, padding='post')

# Convert labels to one-hot encoding
labels = to_categorical(windowed_labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)


# Combine augmented data
# all_combined_data = augmented_combined_bach + augmented_combined_mozart + augmented_combined_beethoven + augmented_combined_chopin


# Combine all data
# all_combined_data = augmented_combined_bach + augmented_combined_mozart + augmented_combined_beethoven + augmented_combined_chopin

# # Combine augmented data and create labels
# all_data = augmented_bach + augmented_mozart + augmented_beethoven + augmented_chopin
# labels = [0] * len(augmented_combined_bach) + [1] * len(augmented_combined_mozart) + \
        #  [2] * len(augmented_combined_beethoven) + [3] * len(augmented_combined_chopin)

# # --- Data Preparation ---

# # Tokenize the note pitches
# all_pitches = sorted(set(note for sequence in all_data for note in sequence))
# note_to_int = {note: number for number, note in enumerate(all_pitches)}
# int_to_note = {number: note for note, number in note_to_int.items()}

# Convert note sequences to integer sequences
# tokenized_data = [[note_to_int[note] for note in sequence] for sequence in all_data]

# Pad sequences
# Update max sequence length for the new data
# max_combined_sequence_length = max([len(seq) for seq in all_combined_data])

# Pad combined sequences
# X_combined = pad_sequences(all_combined_data, maxlen=max_combined_sequence_length, padding='post', dtype='float32')

# Split data into training and testing sets
# X_train_combined, X_test_combined, y_train, y_test = train_test_split(X_combined, labels, test_size=0.2, random_state=42)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.8 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592288 sha256=115e31029828e3045198f22c54676d7b1866792647aede169cfabbdc185f06c7
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


/usr/local/lib/python3.10/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Skipping file Anhang 14-3.mid due to KeySignatureError: Could not decode key with 3 flats and mode 255


In [5]:
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [ ]:
from tensorflow.keras.layers import BatchNormalization

# Create LSTM model
model_combined = Sequential()
model_combined.add(LSTM(256, return_sequences=True, input_shape=(window_size, 3)))  # 3 features
model_combined.add(BatchNormalization())
model_combined.add(Dropout(0.3))
model_combined.add(LSTM(128))
model_combined.add(BatchNormalization())
model_combined.add(Dropout(0.3))
model_combined.add(Dense(64, activation='relu'))
model_combined.add(Dense(4, activation='softmax'))

# Compile the model
model_combined.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model_combined.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model_combined.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/10
445/659 [===================>..........] - ETA: 2:06:45 - loss: 1.3843 - accuracy: 0.3098

In [ ]:
# --- LSTM Model for Windowed Input ---

# Define LSTM model
model_windowed = Sequential()
model_windowed.add(Embedding(len(all_pitches), 128, input_length=window_size))  # Window size as input length
model_windowed.add(LSTM(256, return_sequences=True))
model_windowed.add(Dropout(0.3))
model_windowed.add(LSTM(128))
model_windowed.add(Dropout(0.3))
model_windowed.add(Dense(64, activation='relu'))
model_windowed.add(Dense(4, activation='softmax'))

# Compile the model
model_windowed.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early Stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model_windowed.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model_windowed.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/10
10991/23861 [============>.................] - ETA: 28:00:22 - loss: 0.6643 - accuracy: 0.7198